whole process:
    1. search for the file for the folder
    2. use boto3 to read the file
    3. write into the new folder

In [38]:
import boto3
import io
from PIL import Image
import s3fs
fs = s3fs.S3FileSystem()
s3 = boto3.client('s3', region_name='us-east-2')
import tensorflow as tf
import pandas as pd
from tensorflow.contrib.predictor import from_saved_model
import s3fs

In [39]:
#search images
def search_image(root_path, all_files = []):
    files = fs.ls(root_path) 
    for file in files:
        if not fs.isdir(file):
            all_files.append(file)
        else:
            search_image(file,all_files)
    return all_files

In [40]:
subdirs = fs.ls('s3://...')
subdirs

['ingo-images/all_images/IMAGESTOR-04/Default/TEST00_MRC2-1',
 'ingo-images/all_images/IMAGESTOR-04/Default/TestWin10',
 'ingo-images/all_images/IMAGESTOR-04/Default/TesyWin10',
 'ingo-images/all_images/IMAGESTOR-04/Default/xcxw3-149',
 'ingo-images/all_images/IMAGESTOR-04/Default/xdfp35-079',
 'ingo-images/all_images/IMAGESTOR-04/Default/xdfp40-030',
 'ingo-images/all_images/IMAGESTOR-04/Default/xdhn3-062',
 'ingo-images/all_images/IMAGESTOR-04/Default/xdmf3-006',
 'ingo-images/all_images/IMAGESTOR-04/Default/xdmi1-007',
 'ingo-images/all_images/IMAGESTOR-04/Default/xdmj1-019',
 'ingo-images/all_images/IMAGESTOR-04/Default/xdmj1-020']

In [41]:
subdirs[9][-9:-5]#

'xdmj'

In [42]:
subdirs = fs.ls('s3:...')
absroot = []
for d in subdirs:
    if (d[-9:-5] == 'xdmj'):
        absroot.append(d)
for i in absroot:
    files = search_image(i)

In [43]:
def transfer_files(s3, old_path:tuple, new_path:tuple):
    down_stream = s3.get_object(Bucket=old_path[0],Key=old_path[1])['Body']
    im = Image.open(down_stream)
    up_stream = io.BytesIO()
    im.save(up_stream, format=im.format)
    up_stream.seek(0)
    s3.upload_fileobj(up_stream, new_path[0], new_path[1])
    down_stream.close()
    up_stream.close()
    return 

In [44]:
for old_path in files:
    pos = old_path.find('/')
    old_path = (old_path[:pos],old_path[pos+1:])
    k = old_path[1]
    p = k.rfind('/')
    new_path = ('Bucket Name','Object Nmae'+k[p+1:])
    transfer_files(s3, old_path, new_path)